In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [3]:
import pandas as pd
import numpy as np

In [26]:
df_geom = pd.read_csv('../data/geometry_bestguess.hgcal.txt', sep=' ')
df_constr = pd.read_csv('../data/220712_strobbe_elinks-mapping.csv')
showermax_layers = [7,9,11,13]
hcal_start = 27
elinks_showermax = 4
elinks_regular = 2
# STC: assuming STC4 5E+4M in HDM and STC16 in LDM
elinks_stc16 = 2
elinks_stc4 = 5

In [27]:
print(df_geom.head())
print(df_constr.head())

   plane  u  v itype      x0      y0  irot  nvertices   vx_0   vy_0  ...  \
0      1  2  0   gIe  334.88    0.00     3          6  334.9   96.7  ...   
1      1  2  1   dIe  251.16  145.01     3          5  251.2  241.7  ...   
2      1  3  0    FI  502.32    0.00     0          6  502.3  -96.7  ...   
3      1  3  1    FI  418.60  145.01     1          6  418.6   48.3  ...   
4      1  3  2    FI  334.88  290.02     1          6  334.9  193.3  ...   

   trigPp0_type           dataPp1           trigPp1  dataPp1_type  \
0             A  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_2            Ap   
1             B  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_3            Ap   
2             A  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_2            Ap   
3             A  PP1(CE-)5_FO_1_3  PP1(CE-)3_FO_2_2            Ap   
4             B  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_3            Ap   

   trigPp1_type         dataPp2      DAQ  engine_trig_fibres  \
0            Ap  PPFO(CE-)X5_14  DAQ_2_2                   3   


In [28]:
df_geom = df_geom[['plane','u','v','trigLinks','itype']]
df_constr = df_constr.set_index(['layer', 'u', 'v'])

In [29]:
def get_constr_bcstc(row):
    layer = int(row.plane)
    u = int(row.u)
    v = int(row.v)
    itype = row.itype
    triangle = itype[0]=='c'
    hdm = 'I' in itype
    bcstc_elinks = 0
    constr_elinks = 999
    bcstc_elinks = elinks_showermax if layer in showermax_layers else elinks_regular
    # no trigger links in even layers
    if layer%2==0 and layer<hcal_start: bcstc_elinks = 0 
    # STC elinks: STC4 in HDM and STC16 in LDM
    if layer>=hcal_start:
        if hdm:
            bcstc_elinks = elinks_stc4
        else:
            bcstc_elinks = elinks_stc16
    layer_tmp = min(layer, hcal_start) # all HCAL layers have the same constraints
    if df_constr.index.isin([(layer_tmp, u, v)]).any():
        constr_elinks = int(df_constr.loc[(layer_tmp,u,v)])
    # No trigger elinks in triangles/three-partials
    if triangle: constr_elinks = 0
    return min(bcstc_elinks, constr_elinks)

In [30]:
df_geom['trigLinksBCSTC'] = df_geom.apply(get_constr_bcstc, axis=1)

In [31]:
df_geom['ok'] = df_geom['trigLinks']>=df_geom['trigLinksBCSTC']

In [33]:
df_geom.query('ok==False')

,plane,u,v,trigLinks,itype,trigLinksBCSTC,ok
3484,7,7,7,3.0,FO,4,False
3491,7,8,8,3.0,FO,4,False
3501,7,7,9,3.0,FO,4,False
3510,7,6,10,3.0,bOe,4,False
3583,9,7,7,3.0,FO,4,False
3590,9,8,8,2.0,FO,4,False
3598,9,9,9,1.0,aOeB,4,False
3600,9,7,9,3.0,FO,4,False
3607,9,8,10,3.0,aOeB,4,False
3609,9,6,10,3.0,FOe,4,False
